In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import cv2 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
SEED = 77
IMG_SIZE = 512

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

x = df_train['id_code']
y = df_train['diagnosis']

x, y = shuffle(x, y, random_state=SEED)

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(x, y, test_size=0.15,
                                                      stratify=y, random_state=SEED)
print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape)

In [ ]:
%matplotlib inline

In [ ]:
%%time
fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
def local_avg(img):
    #blurring
    blurred = cv2.blur(img, ksize=(15, 15))
    
    # Take the difference with the original image
    # Weight with a factor of 4x to increase contrast 
    #change '4' to different values to view different preprocessed images
    dst = cv2.addWeighted(img, 4, blurred, -4, 128)
    return dst

In [ ]:
%%time
fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        dst = local_avg(image)
        plt.imshow(dst)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
def equalize_histogram(img):
    dst = img
    for c in range(0, 2):
        dst[:,:,c] = cv2.equalizeHist(dst[:,:,c])
    return dst

In [ ]:
%%time
fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        dst = equalize_histogram(image)
        plt.imshow(dst)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
%%time

#trying local average + histogram equalization

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = local_avg(image)
        dst = equalize_histogram(image)
        plt.imshow(dst)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
%%time

#trying local average + histogram equalization

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        dst = equalize_histogram(image)
        dst = local_avg(dst)
        plt.imshow(dst)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
#ben's preprocessing
###This is a function I used from a super helpful kernel 
###to crop the images to remove the unneccessary black borders.
def crop_image_from_gray(img,tol=7):
    """
    Crop out black borders
    https://www.kaggle.com/ratthachat/aptos-updated-preprocessing-ben-s-cropping
    """  
    
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mask = gray_img>tol        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img


def circle_crop(img):   
    """
    Create circular crop around image centre    
    """    
    #img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
#     img = crop_image_from_gray(img)
    
    return img 

In [ ]:
%%time

#trying local average + histogram equalization

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        dst = circle_crop(image)
        plt.imshow(dst)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
#ben's preprocessing that gives bnw images

def crop_image(img, tol=7):
    mask = img > tol
    return img[np.ix_(mask.any(1), mask.any(0))]

In [ ]:
%%time

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image = crop_image(image)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), IMG_SIZE/10), -4, 128)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
%%time

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = crop_image(image)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), IMG_SIZE/10), -4, 128)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
#clahe algorithm

def clahe_rgb(img):
#     bgr = cv.imread(input_path)
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    gridsize = 5
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(gridsize,gridsize))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    bgr2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
#     cv.imshow('CLAHE RGB', bgr2)
    return bgr2

In [ ]:
#visualizing clahe
path = f"../input/test_images/270a532df702.png"
img = cv2.imread(path)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.subplot(121), plt.imshow(img)
dst = clahe_rgb(img)
plt.subplot(122), plt.imshow(dst)
plt.show()

In [ ]:
%%time

#trying local average + histogram equalization

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = clahe_rgb(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )


In [ ]:
#normalizing images

def normalize_img(img):
    normalized_img = np.zeros((800, 800))
    normalized_img = cv2.normalize(img,  normalized_img, 0, 255, cv2.NORM_MINMAX)
    return normalized_img

In [ ]:
%%time

#trying local average + histogram equalization

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = normalize_img(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [ ]:
%%time

#trying circle crop

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = circle_crop(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying local average + circle crop

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = local_avg(image)
        image = circle_crop(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
def clahe_is_lit(img):   
    bgr = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)    
    lab_planes = cv2.split(bgr)    
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))    
    lab_planes[0] = clahe.apply(lab_planes[0])   
    lab = cv2.merge(lab_planes)
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    return rgb

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
#         image = load_ben_color(path)
        image = cv2.imread(path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = circle_crop(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
def clahe(img):   
#     bgr = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)    
    lab_planes = cv2.split(img)    
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    lab_planes[1] = clahe.apply(lab_planes[1])   #clahe on green channel
    res_img = cv2.merge(lab_planes)
    bgr = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)   
    planes = cv2.split(bgr)
    planes[0] = clahe.apply(planes[0])
    lab = cv2.merge(planes)
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    return rgb

def preprocessing(img):
    img = clahe(img)
    return img

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = circle_crop(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = local_avg(image)
        image = circle_crop(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = local_avg(image)
        image = circle_crop(image)
        image = cv2.medianBlur(image, 9)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
def clahe_gray(img):
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))    
    img = clahe.apply(img)
    return clahe_gray

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = local_avg(image)
        image = circle_crop(image)
        image = bv(image)
        image = cv2.equalizeHist(image)
        image = cv2.medianBlur(image, 9)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = local_avg(image)
        image = circle_crop(image)
        image = bv(image)
        image = cv2.equalizeHist(image)
        image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image , (0,0) , IMG_SIZE/10) ,-4 ,128)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = local_avg(image)
        image = circle_crop(image)
        image = bv(image)
        image = cv2.equalizeHist(image)
        image = cv2.addWeighted(image, 3, cv2.GaussianBlur(image , (0,0) , IMG_SIZE/10) ,-4 ,128)
        image = cv2.medianBlur(image, 9)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
def green_comp(img):
    green = img[:,:,1]
    return green

In [ ]:
def histEqualize(img):
    histEqImg = cv2.equalizeHist(img)
    return histEqImg 

In [ ]:
    def applyKirschFilter(gray):
        if gray.ndim > 2:
            raise Exception("illegal argument: input must be a single channel image (gray)")
        kernelG1 = np.array([[ 5,  5,  5],
                             [-3,  0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG2 = np.array([[ 5,  5, -3],
                             [ 5,  0, -3],
                             [-3, -3, -3]], dtype=np.float32)
        kernelG3 = np.array([[ 5, -3, -3],
                             [ 5,  0, -3],
                             [ 5, -3, -3]], dtype=np.float32)
        kernelG4 = np.array([[-3, -3, -3],
                             [ 5,  0, -3],
                             [ 5,  5, -3]], dtype=np.float32)
        kernelG5 = np.array([[-3, -3, -3],
                             [-3,  0, -3],
                             [ 5,  5,  5]], dtype=np.float32)
        kernelG6 = np.array([[-3, -3, -3],
                             [-3,  0,  5],
                             [-3,  5,  5]], dtype=np.float32)
        kernelG7 = np.array([[-3, -3,  5],
                             [-3,  0,  5],
                             [-3, -3,  5]], dtype=np.float32)
        kernelG8 = np.array([[-3,  5,  5],
                             [-3,  0,  5],
                             [-3, -3, -3]], dtype=np.float32)
    
        g1 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG1), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g2 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG2), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g3 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG3), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g4 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG4), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g5 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG5), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g6 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG6), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g7 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG7), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        g8 = cv2.normalize(cv2.filter2D(gray, cv2.CV_32F, kernelG8), None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
        magn = cv2.max(g1, cv2.max(g2, cv2.max(g3, cv2.max(g4, cv2.max(g5, cv2.max(g6, cv2.max(g7, g8)))))))
        return magn

In [ ]:
def applyThreshold(img):
    ret, threshImg = cv2.threshold(img,160,180,cv2.THRESH_BINARY_INV)
    return threshImg

In [ ]:
def cleanSmallObjects(img):
    cleanImg = morphology.remove_small_objects(img, min_size=130, connectivity=100)
    return cleanImg

In [ ]:
def pp(img):
    img = green_comp(img)
    img = histEqualize(img)
    img = applyKirschFilter(img)
    img = applyThreshold(img)
#     img = cleanSmallObjects(img)
    return img

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = pp(image)
#         image = circle_crop(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
def bv(image):
	b,green_fundus,r = cv2.split(image)
	clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
	contrast_enhanced_green_fundus = clahe.apply(green_fundus)

	# applying alternate sequential filtering (3 times closing opening)
	r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
	R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
	r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
	R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
	r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
	R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)	
	f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
	f5 = clahe.apply(f4)
    
	return f5

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = circle_crop(image)
        image = clahe_is_lit(image)
        image = bv(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = circle_crop(image)
        image = bv(image)
        image = cv2.equalizeHist(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = circle_crop(image)
        image = bv(image)
        image = cv2.equalizeHist(image)
        image = cv2.medianBlur(image, 9)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = circle_crop(image)
        image = bv(image)
        image = cv2.equalizeHist(image)
        image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), IMG_SIZE/10), -4, 128)
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#         image = cv2.inRange(image,(40, 5, 30), (50, 100, 255) )

        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = circle_crop(image)
        image = bv(image)
        image = cv2.equalizeHist(image)
        image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), IMG_SIZE/10), -4, 128)
        image = cv2.medianBlur(image,9)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )

In [ ]:
%%time

#trying clahe + removing gray color

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = circle_crop(image)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )
    

In [ ]:
image[100][100]

In [ ]:
print(image[150][100])
print(image[200][200])
print(image[256][256])

In [ ]:
%%time

#trying clahe + removing gray color

gray = np.array([140,140,140])

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = circle_crop(image)
        white = [140,140,140]
        black = [0,0,0]
        for x in range(0,512):
            for y in range(0,512):
                channels_xy = image[y,x]
                if all(channels_xy >= [108,108,108]) and all(channels_xy<=[160,160,160]):
                    image[y,x] = [255,255,255]
#                 elif all(channels_xy == black):
#                     img[y,x] = white
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']) )
    

In [ ]:
%%time

#trying clahe + removing gray color

gray = np.array([140,140,140])

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = load_ben_color(path)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = clahe_is_lit(image)
        image = circle_crop(image)
        white = [140,140,140]
        black = [0,0,0]
        for x in range(0,512):
            for y in range(0,512):
                channels_xy = image[y,x]
                if all(channels_xy >= [100,100,100]) and all(channels_xy<=[180,180,180]):
                    image[y,x] = [255,255,255]
#                 elif all(channels_xy == black):
#                     img[y,x] = white
        image = cv2.medianBlur(image, 3)
        plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']))

**PREPROCESSING STARTS FROM HERE NEW**

In [ ]:
def blood_vessel(image):
	b,green_fundus,r = cv2.split(image)
	clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
	contrast_enhanced_green_fundus = clahe.apply(green_fundus)

	# applying alternate sequential filtering (3 times closing opening)
	r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
	R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)), iterations = 1)
	r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
	R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11)), iterations = 1)
	r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
	R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(23,23)), iterations = 1)
	f4 = cv2.subtract(R3,contrast_enhanced_green_fundus)
	f5 = clahe.apply(f4)

# 	f5 = cv2.merge((b,f5,r))
# 	removing very small contours through area parameter noise removal
	ret,f6 = cv2.threshold(f5,15,255,cv2.THRESH_BINARY)
	mask = np.ones(f5.shape[:2], dtype="uint8") * 255
	contours, hierarchy = cv2.findContours(f6,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
	for cnt in contours:
		if cv2.contourArea(cnt) <= 200:
			cv2.drawContours(mask, [cnt], -1, 0, -1)
	im = cv2.bitwise_and(f5, f5, mask=mask)
	ret,fin = cv2.threshold(im,15,255,cv2.THRESH_BINARY_INV)
	newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)

# 	removing blobs of unwanted bigger chunks taking in consideration they are not straight lines like blood
# 	vessels and also in an interval of area
	fundus_eroded = cv2.bitwise_not(f5)
	xmask = np.ones(image.shape[:2], dtype="uint8") * 255
	xcontours, xhierarchy = cv2.findContours(fundus_eroded,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
	for cnt in xcontours:
		shape = "unidentified"
		peri = cv2.arcLength(cnt, True)
		approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)
		if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
			shape = "circle"
		else:
			shape = "veins"
		if(shape=="circle"):
			cv2.drawContours(xmask, [cnt], -1, 0, -1)

	finimage = cv2.bitwise_and(fundus_eroded,fundus_eroded,mask=xmask)
	blood_vessels = cv2.bitwise_not(finimage)
	return blood_vessels

In [ ]:
%%time

#trying clahe + removing gray color

gray = np.array([140,140,140])

fig = plt.figure(figsize=(25, 16))
# display 10 images from each class
for class_id in sorted(train_y.unique()):
    for i, (idx, row) in enumerate(df_train.loc[df_train['diagnosis'] == class_id].sample(5, random_state=SEED).iterrows()):
        ax = fig.add_subplot(5, 5, class_id * 5 + i + 1, xticks=[], yticks=[])
        path=f"../input/train_images/{row['id_code']}.png"
        image = cv2.imread(path)
        image = blood_vessel(image)
        image = cv2.resize(image, (224,224))
#         for i in range(0,224):
#             for j in range(0,224):
#                 if image[i][j] >=15:
#                     image[i][j] = 255
#         plt.imshow(image)
        ax.set_title('Label: %d-%d-%s' % (class_id, idx, row['id_code']))